In [162]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.python.framework import ops
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
ops.reset_default_graph()
sess = tf.InteractiveSession()

# Se cargan los datos

In [163]:
#sess = tf.Session()# Training loop
Xdf_raw = pd.read_csv("../IndoorFingerprint.csv").drop("Y", axis=1)
Ydf_raw = pd.read_csv("../IndoorFingerprint.csv").drop("X", axis=1)
x = tf.placeholder(tf.float32, shape=[None, 8])
y_ = tf.placeholder(tf.float32, shape=[None, 11])

# Se escalan los datos

In [164]:
# Normalizacion de los datos
atribute_numbers = Xdf_raw.shape[1]-1

# Valores para el dataset que predice X
Xtrx, Xtex, ytrx, ytex = train_test_split(
Xdf_raw.iloc[:, 1:], Xdf_raw["X"], test_size=0.2, random_state=0)
train_size = len(Xtrx)
test_size = len(Xtex)

# Valores para el dataset que predice Y
Xtry, Xtey, ytry, ytey = train_test_split(
Ydf_raw.iloc[:, 1:], Ydf_raw["Y"], test_size=0.2, random_state=0)

# Se normalizan los datos para ambos

scaler = preprocessing.StandardScaler()
Xtrx = scaler.fit_transform(Xtrx)
Xtex = scaler.transform(Xtex)

scaler = preprocessing.StandardScaler()
Xtry = scaler.fit_transform(Xtry)
Xtey = scaler.transform(Xtey)

# One Hot Vector

In [165]:
class_number_x = ytrx.unique().size
class_number_y = ytry.unique().size

ytrx_dummy = pd.get_dummies(ytrx)
ytex_dummy = pd.get_dummies(ytex)

ytry_dummy = pd.get_dummies(ytry)
ytey_dummy = pd.get_dummies(ytey)

# Se convierten valores a Tensores X

Xtrx = np.array(Xtrx,dtype='float32')
Xtex = np.array(Xtex,dtype='float32')
ytrx_dummy = np.array(ytrx_dummy,dtype='int32')
ytex_dummy = np.array(ytex_dummy,dtype='int32')

# Se convierten valores a Tensores Y

Xtry = np.array(Xtry,dtype='float32')
Xtey = np.array(Xtey,dtype='float32')
ytry_dummy = np.array(ytry_dummy,dtype='int32')
ytey_dummy = np.array(ytey_dummy,dtype='int32')

ytrx_cat = pd.Series(ytrx, dtype="category")
ytrx = np.array(ytrx_cat.cat.codes,dtype='int32')
ytex_cat = pd.Series(ytex, dtype="category")
ytex = np.array(ytex_cat.cat.codes,dtype='int32')

print(Xtrx.shape)
print(ytrx_dummy.shape)

(5280, 8)
(5280, 11)


In [166]:
x = tf.placeholder(tf.float32, shape=[None, atribute_numbers])
y_ = tf.placeholder(tf.float32, shape=[None, class_number_x])

In [167]:
W = tf.Variable(tf.zeros([atribute_numbers,class_number_x]))
b = tf.Variable(tf.zeros([class_number_x]))

In [168]:
sess.run(tf.global_variables_initializer())

In [169]:
y = tf.matmul(x,W) + b

In [170]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [171]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
batch_size = 32

In [172]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [173]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [174]:
for _ in range(1000):
    #for batch in iterate_minibatches(Xtrx, ytrx_dummy, batch_size, shuffle=True):
    batch = next_batch(batch_size, Xtrx, ytrx_dummy)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [175]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: Xtex, y_: ytex_dummy}))

0.699242


# CNN

In [176]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

'''def get_iris_data():
    """ Read the iris data set and split them into training and test sets """
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]

    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data

    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)'''

def runTraining():

    # Layer's sizes
    #x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 500           # Number of hidden nodes
    #y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, atribute_numbers])
    y = tf.placeholder("float", shape=[None, class_number_x])

    # Weight initializations
    w_1 = init_weights((atribute_numbers, h_size))
    w_2 = init_weights((h_size, class_number_x))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(1000):
        # Train with each example
        batch = next_batch(batch_size, Xtrx, ytrx_dummy)
        sess.run(updates, feed_dict={X: batch[0], y: batch[1]})

        train_accuracy = np.mean(np.argmax(ytrx_dummy, axis=1) ==
                                 sess.run(predict, feed_dict={X: Xtrx, y: ytrx_dummy}))
        test_accuracy  = np.mean(np.argmax(ytex_dummy, axis=1) ==
                                 sess.run(predict, feed_dict={X: Xtex, y: ytex_dummy}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()

In [177]:
runTraining()

Epoch = 1, train accuracy = 9.11%, test accuracy = 8.94%
Epoch = 2, train accuracy = 7.16%, test accuracy = 7.20%
Epoch = 3, train accuracy = 7.23%, test accuracy = 7.20%
Epoch = 4, train accuracy = 5.57%, test accuracy = 5.76%
Epoch = 5, train accuracy = 6.21%, test accuracy = 7.12%
Epoch = 6, train accuracy = 5.91%, test accuracy = 7.12%
Epoch = 7, train accuracy = 8.30%, test accuracy = 8.18%
Epoch = 8, train accuracy = 9.13%, test accuracy = 8.94%
Epoch = 9, train accuracy = 7.71%, test accuracy = 7.73%
Epoch = 10, train accuracy = 6.95%, test accuracy = 7.88%
Epoch = 11, train accuracy = 5.23%, test accuracy = 5.45%
Epoch = 12, train accuracy = 4.62%, test accuracy = 4.62%
Epoch = 13, train accuracy = 4.56%, test accuracy = 4.55%
Epoch = 14, train accuracy = 4.55%, test accuracy = 4.55%
Epoch = 15, train accuracy = 5.49%, test accuracy = 4.92%
Epoch = 16, train accuracy = 7.77%, test accuracy = 7.80%
Epoch = 17, train accuracy = 7.31%, test accuracy = 7.35%
Epoch = 18, train accur

Epoch = 140, train accuracy = 26.19%, test accuracy = 27.05%
Epoch = 141, train accuracy = 25.32%, test accuracy = 25.98%
Epoch = 142, train accuracy = 25.98%, test accuracy = 26.67%
Epoch = 143, train accuracy = 26.50%, test accuracy = 27.20%
Epoch = 144, train accuracy = 27.67%, test accuracy = 27.73%
Epoch = 145, train accuracy = 28.26%, test accuracy = 27.35%
Epoch = 146, train accuracy = 27.18%, test accuracy = 27.88%
Epoch = 147, train accuracy = 26.69%, test accuracy = 28.03%
Epoch = 148, train accuracy = 27.29%, test accuracy = 27.58%
Epoch = 149, train accuracy = 25.38%, test accuracy = 26.29%
Epoch = 150, train accuracy = 20.89%, test accuracy = 22.12%
Epoch = 151, train accuracy = 22.46%, test accuracy = 24.55%
Epoch = 152, train accuracy = 21.57%, test accuracy = 22.88%
Epoch = 153, train accuracy = 22.23%, test accuracy = 24.32%
Epoch = 154, train accuracy = 23.73%, test accuracy = 25.83%
Epoch = 155, train accuracy = 23.43%, test accuracy = 25.98%
Epoch = 156, train accur

Epoch = 276, train accuracy = 30.91%, test accuracy = 32.12%
Epoch = 277, train accuracy = 31.25%, test accuracy = 33.03%
Epoch = 278, train accuracy = 32.25%, test accuracy = 33.33%
Epoch = 279, train accuracy = 35.25%, test accuracy = 37.27%
Epoch = 280, train accuracy = 33.58%, test accuracy = 34.24%
Epoch = 281, train accuracy = 32.69%, test accuracy = 32.35%
Epoch = 282, train accuracy = 32.56%, test accuracy = 33.11%
Epoch = 283, train accuracy = 33.24%, test accuracy = 33.86%
Epoch = 284, train accuracy = 36.10%, test accuracy = 35.30%
Epoch = 285, train accuracy = 37.77%, test accuracy = 36.44%
Epoch = 286, train accuracy = 37.54%, test accuracy = 36.74%
Epoch = 287, train accuracy = 31.33%, test accuracy = 31.21%
Epoch = 288, train accuracy = 33.37%, test accuracy = 33.64%
Epoch = 289, train accuracy = 32.41%, test accuracy = 31.97%
Epoch = 290, train accuracy = 34.83%, test accuracy = 33.18%
Epoch = 291, train accuracy = 36.33%, test accuracy = 35.68%
Epoch = 292, train accur

Epoch = 415, train accuracy = 34.22%, test accuracy = 33.41%
Epoch = 416, train accuracy = 35.42%, test accuracy = 34.47%
Epoch = 417, train accuracy = 40.08%, test accuracy = 39.70%
Epoch = 418, train accuracy = 43.75%, test accuracy = 43.48%
Epoch = 419, train accuracy = 39.56%, test accuracy = 38.56%
Epoch = 420, train accuracy = 40.83%, test accuracy = 39.92%
Epoch = 421, train accuracy = 40.89%, test accuracy = 39.32%
Epoch = 422, train accuracy = 37.92%, test accuracy = 37.27%
Epoch = 423, train accuracy = 40.47%, test accuracy = 39.17%
Epoch = 424, train accuracy = 44.05%, test accuracy = 41.82%
Epoch = 425, train accuracy = 40.11%, test accuracy = 37.73%
Epoch = 426, train accuracy = 43.37%, test accuracy = 41.59%
Epoch = 427, train accuracy = 36.82%, test accuracy = 34.62%
Epoch = 428, train accuracy = 39.02%, test accuracy = 35.53%
Epoch = 429, train accuracy = 39.38%, test accuracy = 37.12%
Epoch = 430, train accuracy = 37.48%, test accuracy = 35.15%
Epoch = 431, train accur

Epoch = 553, train accuracy = 52.59%, test accuracy = 52.12%
Epoch = 554, train accuracy = 51.21%, test accuracy = 51.14%
Epoch = 555, train accuracy = 46.91%, test accuracy = 47.50%
Epoch = 556, train accuracy = 40.72%, test accuracy = 41.06%
Epoch = 557, train accuracy = 47.16%, test accuracy = 46.89%
Epoch = 558, train accuracy = 47.63%, test accuracy = 48.56%
Epoch = 559, train accuracy = 43.14%, test accuracy = 44.39%
Epoch = 560, train accuracy = 41.57%, test accuracy = 42.95%
Epoch = 561, train accuracy = 46.14%, test accuracy = 46.74%
Epoch = 562, train accuracy = 47.37%, test accuracy = 45.23%
Epoch = 563, train accuracy = 48.28%, test accuracy = 45.23%
Epoch = 564, train accuracy = 43.94%, test accuracy = 44.39%
Epoch = 565, train accuracy = 39.73%, test accuracy = 40.15%
Epoch = 566, train accuracy = 43.26%, test accuracy = 44.32%
Epoch = 567, train accuracy = 40.34%, test accuracy = 42.35%
Epoch = 568, train accuracy = 36.16%, test accuracy = 38.94%
Epoch = 569, train accur

Epoch = 688, train accuracy = 42.14%, test accuracy = 43.71%
Epoch = 689, train accuracy = 44.73%, test accuracy = 46.21%
Epoch = 690, train accuracy = 44.94%, test accuracy = 47.20%
Epoch = 691, train accuracy = 42.54%, test accuracy = 44.77%
Epoch = 692, train accuracy = 46.40%, test accuracy = 48.48%
Epoch = 693, train accuracy = 46.70%, test accuracy = 50.30%
Epoch = 694, train accuracy = 48.35%, test accuracy = 50.83%
Epoch = 695, train accuracy = 48.79%, test accuracy = 50.30%
Epoch = 696, train accuracy = 52.08%, test accuracy = 53.56%
Epoch = 697, train accuracy = 53.35%, test accuracy = 54.77%
Epoch = 698, train accuracy = 53.12%, test accuracy = 54.70%
Epoch = 699, train accuracy = 50.42%, test accuracy = 51.29%
Epoch = 700, train accuracy = 49.03%, test accuracy = 48.11%
Epoch = 701, train accuracy = 46.02%, test accuracy = 45.23%
Epoch = 702, train accuracy = 46.50%, test accuracy = 44.39%
Epoch = 703, train accuracy = 46.78%, test accuracy = 47.12%
Epoch = 704, train accur

Epoch = 823, train accuracy = 45.30%, test accuracy = 47.50%
Epoch = 824, train accuracy = 49.19%, test accuracy = 50.38%
Epoch = 825, train accuracy = 49.17%, test accuracy = 50.08%
Epoch = 826, train accuracy = 47.23%, test accuracy = 49.32%
Epoch = 827, train accuracy = 44.77%, test accuracy = 45.98%
Epoch = 828, train accuracy = 44.60%, test accuracy = 46.52%
Epoch = 829, train accuracy = 40.57%, test accuracy = 41.36%
Epoch = 830, train accuracy = 39.00%, test accuracy = 40.68%
Epoch = 831, train accuracy = 37.95%, test accuracy = 38.64%
Epoch = 832, train accuracy = 39.94%, test accuracy = 40.98%
Epoch = 833, train accuracy = 40.21%, test accuracy = 41.97%
Epoch = 834, train accuracy = 42.67%, test accuracy = 45.30%
Epoch = 835, train accuracy = 44.94%, test accuracy = 46.21%
Epoch = 836, train accuracy = 47.41%, test accuracy = 48.41%
Epoch = 837, train accuracy = 47.44%, test accuracy = 48.86%
Epoch = 838, train accuracy = 51.42%, test accuracy = 51.29%
Epoch = 839, train accur

Epoch = 960, train accuracy = 52.75%, test accuracy = 53.18%
Epoch = 961, train accuracy = 53.79%, test accuracy = 54.02%
Epoch = 962, train accuracy = 54.05%, test accuracy = 52.58%
Epoch = 963, train accuracy = 53.12%, test accuracy = 51.67%
Epoch = 964, train accuracy = 54.41%, test accuracy = 53.41%
Epoch = 965, train accuracy = 53.09%, test accuracy = 52.12%
Epoch = 966, train accuracy = 52.29%, test accuracy = 51.29%
Epoch = 967, train accuracy = 50.89%, test accuracy = 50.76%
Epoch = 968, train accuracy = 51.44%, test accuracy = 51.36%
Epoch = 969, train accuracy = 49.26%, test accuracy = 49.17%
Epoch = 970, train accuracy = 50.25%, test accuracy = 48.71%
Epoch = 971, train accuracy = 46.93%, test accuracy = 45.23%
Epoch = 972, train accuracy = 45.02%, test accuracy = 42.88%
Epoch = 973, train accuracy = 45.95%, test accuracy = 43.56%
Epoch = 974, train accuracy = 44.02%, test accuracy = 42.27%
Epoch = 975, train accuracy = 44.85%, test accuracy = 43.18%
Epoch = 976, train accur